# DATA EXPLORATION NOTEBOOK

### **I. Import thư viện và tải dữ liệu**

**Import thư viện**

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import normaltest, shapiro, pearsonr, spearmanr
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
import warnings
import os
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis
import warnings
warnings.filterwarnings('ignore')

**Cấu hình**

In [11]:
sns.set_style("whitegrid")
sns.set_palette("husl")
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 10

INPUT_FOLDER = "input"
INPUT_FILE = "football_players_dataset.csv"

**Lấy dữ liệu từ file**

In [12]:
file_path = os.path.join(INPUT_FOLDER, INPUT_FILE)
df = pd.read_csv(file_path)

print(f"\nKích thước Dataset:")
print(f"   - Số lượng bản ghi (dòng): {len(df):,}")
print(f"   - Số lượng thuộc tính (cột): {len(df.columns)}")
print(f"   - Tổng số ô dữ liệu: {df.shape[0] * df.shape[1]:,}")
print(f"   - Dung lượng bộ nhớ: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

print("\n10 dòng đầu tiên")
df.head(10)


Kích thước Dataset:
   - Số lượng bản ghi (dòng): 14,977
   - Số lượng thuộc tính (cột): 42
   - Tổng số ô dữ liệu: 629,034
   - Dung lượng bộ nhớ: 7.43 MB

10 dòng đầu tiên


,age,nationality,current_club,league,appearances,minutes_played,is_DF,is_MF,is_FW,goals,...,touches_att_third_per90,touches_att_pen_per90,passes_received_per90,interceptions_per90,blocks_per90,ball_recoveries_per90,aerials_won_per90,yellow_cards_per90,fouls_committed_per90,market_value
0,27.0,Uruguay,Auckland FC,A-League Men,26.0,2268.0,0.0,0.0,1.0,9.0,...,17.61,3.07,28.28,0.48,0.81,3.55,2.59,0.00,2.10,0.75
1,31.0,New Zealand,Auckland FC,A-League Men,26.0,2198.0,1.0,0.0,0.0,2.0,...,13.71,1.49,31.25,0.79,1.02,4.14,1.10,0.19,1.19,0.60
2,26.0,New Zealand,Auckland FC,A-League Men,25.0,2142.0,1.0,0.0,0.0,4.0,...,13.71,1.49,31.25,0.79,1.02,4.14,1.10,0.19,1.19,0.45
3,35.0,Japan,Auckland FC,A-League Men,23.0,2024.0,1.0,0.0,0.0,3.0,...,12.33,0.33,38.17,2.00,1.33,3.00,2.83,0.17,1.00,0.50
4,27.0,New Zealand,Auckland FC,A-League Men,26.0,1782.0,0.0,1.0,0.0,7.0,...,13.71,1.49,31.25,0.79,1.02,4.14,1.10,0.19,1.19,0.45
5,26.0,Belgium,Auckland FC,A-League Men,23.0,1734.0,0.0,1.0,0.0,2.0,...,13.71,2.26,32.26,2.10,1.61,7.10,0.81,0.32,2.26,0.60
6,27.0,Australia,Auckland FC,A-League Men,25.0,1498.0,0.0,1.0,0.0,3.0,...,13.71,1.49,31.25,0.79,1.02,4.14,1.10,0.19,1.19,0.60
7,35.0,New Zealand,Auckland FC,A-League Men,24.0,1213.0,1.0,0.0,0.0,0.0,...,2.16,1.60,23.27,0.78,1.38,4.33,2.28,0.11,0.60,0.10
8,23.0,New Zealand,Auckland FC,A-League Men,13.0,939.0,0.0,0.0,1.0,0.0,...,13.71,1.49,31.25,0.79,1.02,4.14,1.10,0.19,1.19,0.35
9,25.0,New Zealand,Auckland FC,A-League Men,19.0,799.0,0.0,0.0,1.0,2.0,...,13.71,1.49,31.25,0.79,1.02,4.14,1.10,0.19,1.19,0.30


### **II. Khám phá dữ liệu**

#### **1. Ý nghĩa từng cột dữ liệu**

| STT | Cột                              | Ý nghĩa                                                      |
|-----|----------------------------------|--------------------------------------------------------------|
| 1   | age                              | Tuổi hiện tại của cầu thủ                                     |
| 2   | nationality                      | Quốc tịch của cầu thủ                                         |
| 3   | current_club                      | Câu lạc bộ hiện tại                                          |
| 4   | league                            | Giải đấu mà cầu thủ thi đấu                                   |
| 5   | appearances                       | Số trận đã thi đấu                                           |
| 6   | minutes_played                    | Tổng số phút đã thi đấu                                       |
| 7   | is_DF                             | Chỉ số nhị phân hậu vệ                                        |
| 8   | is_MF                             | Chỉ số nhị phân tiền vệ                                       |
| 9   | is_FW                             | Chỉ số nhị phân tiền đạo                                      |
| 10  | goals                             | Tổng số bàn thắng                                            |
| 11  | assists                           | Tổng số kiến tạo                                             |
| 12  | goals_per_90                      | Bàn thắng mỗi 90 phút                                         |
| 13  | npg_per90                         | Bàn thắng không tính penalty mỗi 90 phút                      |
| 14  | xag_per90                         | Expected assisted goals mỗi 90 phút                            |
| 15  | xg_per90                          | Expected goals mỗi 90 phút                                     |
| 16  | shots_per90                       | Số cú sút mỗi 90 phút                                         |
| 17  | shots_on_target_per90             | Số cú sút trúng đích mỗi 90 phút                               |
| 18  | shots_on_target_pct               | Tỷ lệ cú sút trúng đích                                       |
| 19  | avg_shot_distance                 | Khoảng cách trung bình của cú sút                               |
| 20  | sca_per90                         | Hành động tạo cơ hội sút mỗi 90 phút                            |
| 21  | gca_per90                         | Hành động tạo cơ hội ghi bàn mỗi 90 phút                        |
| 22  | key_passes_per90                  | Số đường chuyền quan trọng mỗi 90 phút                          |
| 23  | passes_completed_per90            | Số đường chuyền thành công mỗi 90 phút                          |
| 24  | pass_completion_pct               | Tỷ lệ chuyền bóng thành công                                    |
| 25  | passes_into_final_third_per90     | Số đường chuyền vào 1/3 sân cuối mỗi 90 phút                    |
| 26  | passes_into_penalty_area_per90    | Số đường chuyền vào khu vực 16m50 mỗi 90 phút                    |
| 27  | progressive_passes_per90          | Số đường chuyền tiến triển mỗi 90 phút                           |
| 28  | progressive_passes_rec_per90      | Số đường chuyền tiến triển nhận được mỗi 90 phút                |
| 29  | progressive_carries_per90         | Số lần dẫn bóng tiến triển mỗi 90 phút                           |
| 30  | take_ons_per90                    | Số lần đi bóng qua người mỗi 90 phút                              |
| 31  | carries_per90                     | Số lần dẫn bóng mỗi 90 phút (không bao gồm carries_into_final_third_per90)                                       |
| 32  | carries_into_final_third_per90    | Số lần dẫn bóng vào 1/3 sân cuối mỗi 90 phút                      |
| 33  | touches_att_third_per90           | Số lần chạm bóng trong 1/3 sân cuối mỗi 90 phút                   |
| 34  | touches_att_pen_per90             | Số lần chạm bóng trong khu vực 16m50 mỗi 90 phút                   |
| 35  | passes_received_per90             | Số lần nhận bóng mỗi 90 phút                                      |
| 36  | interceptions_per90               | Số lần cắt bóng mỗi 90 phút                                       |
| 37  | blocks_per90                      | Số lần chặn bóng mỗi 90 phút                                       |
| 38  | ball_recoveries_per90             | Số lần giành lại bóng mỗi 90 phút                                  |
| 39  | aerials_won_per90                 | Số lần thắng tranh chấp trên không mỗi 90 phút                     |
| 40  | yellow_cards_per90                | Số thẻ vàng nhận được mỗi 90 phút                                   |
| 41  | fouls_committed_per90             | Số lỗi phạm phải mỗi 90 phút                                        |
| 42  | calculated_mpg                    | Số phút trung bình mỗi trận                                         |
| 43  | calculated_sum                    | Trường tính tổng (kết hợp các chỉ số)                              |
| 44  | market_value                      | Giá trị thị trường của cầu thủ (triệu €)                             |

#### **2. Kiểu dữ liệu từng cột**

In [13]:
dtype_summary = pd.DataFrame({
    'Cột': df.columns,
    'Kiểu_dữ_liệu': df.dtypes.astype(str).values,
    'Không_null': df.notna().sum().values,
    'Số_null': df.isna().sum().values,
    'Null_%': (df.isna().sum().values / len(df) * 100).round(2),
    'Giá_trị_duy_nhất': [df[col].nunique() for col in df.columns],
    'Mẫu': [str(df[col].dropna().iloc[0])[:30] if df[col].notna().any() else 'N/A' 
            for col in df.columns]
})

print("\nTỔNG HỢP CHI TIẾT KIỂU DỮ LIỆU:")
print("-" * 90)
print(dtype_summary.to_string(index=False))

numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()


TỔNG HỢP CHI TIẾT KIỂU DỮ LIỆU:
------------------------------------------------------------------------------------------
                           Cột Kiểu_dữ_liệu  Không_null  Số_null  Null_%  Giá_trị_duy_nhất          Mẫu
                           age      float64       14977        0     0.0                29         27.0
                   nationality       object       14977        0     0.0               162      Uruguay
                  current_club       object       14977        0     0.0               811  Auckland FC
                        league       object       14977        0     0.0                57 A-League Men
                   appearances      float64       14977        0     0.0                50         26.0
                minutes_played      float64       14977        0     0.0              3063       2268.0
                         is_DF      float64       14977        0     0.0                 2          0.0
                         is_MF      float64 

#### **3. Phân bố kiểu dữ liệu**

In [14]:
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
boolean_cols = df.select_dtypes(include=['bool']).columns.tolist()

print(f"\n1. Cột dạng số: {len(numeric_cols)}")
for i, col in enumerate(numeric_cols, 1):
    dtype = df[col].dtype
    print(f"   {i:2d}. {col:45s} ({dtype})")

print(f"\n2. Cột dạng phân loại: {len(categorical_cols)}")
for i, col in enumerate(categorical_cols, 1):
    n_unique = df[col].nunique()
    print(f"   {i:2d}. {col:45s} ({n_unique} giá trị duy nhất)")


1. Cột dạng số: 39
    1. age                                           (float64)
    2. appearances                                   (float64)
    3. minutes_played                                (float64)
    4. is_DF                                         (float64)
    5. is_MF                                         (float64)
    6. is_FW                                         (float64)
    7. goals                                         (float64)
    8. assists                                       (float64)
    9. goals_per_90                                  (float64)
   10. npg_per90                                     (float64)
   11. xag_per90                                     (float64)
   12. npxg_xag_per90                                (float64)
   13. xg_per90                                      (float64)
   14. shots_per90                                   (float64)
   15. shots_on_target_per90                         (float64)
   16. shots_on_target_pct         

#### **4. Kiểm tra kiểu dữ liệu không phù hợp**

In [15]:
inappropriate_issues = []

# 1. Chỉ số nhị phân
binary_cols = ['is_DF', 'is_MF', 'is_FW']
for col in binary_cols:
    if col in df.columns:
        if df[col].dtype not in ['int64', 'int32']:
            inappropriate_issues.append(f"{col}: Nên là int, hiện tại là {df[col].dtype}")
            print(f"!  {col:40s}: Nên là int, hiện tại là {df[col].dtype}")
        
        unique_vals = df[col].dropna().unique()
        if not all(v in [0, 1] for v in unique_vals):
            inappropriate_issues.append(f"{col}: Chứa giá trị không phải nhị phân: {unique_vals}")
            print(f"x {col:40s}: Chứa giá trị không phải nhị phân!")

# 2. Cột phần trăm
percentage_cols = [col for col in df.columns if 'pct' in col.lower()]
for col in percentage_cols:
    if df[col].dtype in ['int64', 'float64']:
        min_val = df[col].min()
        max_val = df[col].max()
        if min_val < 0 or max_val > 100:
            inappropriate_issues.append(f"{col}: Ngoài khoảng 0-100% (min={min_val:.1f}, max={max_val:.1f})")
            print(f"!  {col:40s}: Ngoài khoảng 0-100% (min={min_val:.1f}, max={max_val:.1f})")

# 3. Số lưu dưới dạng text
for col in categorical_cols:
    if col not in ['nationality', 'current_club', 'league']:
        try:
            numeric_test = pd.to_numeric(df[col], errors='coerce')
            conversion_rate = numeric_test.notna().sum() / df[col].notna().sum() * 100
            
            if conversion_rate > 90:
                inappropriate_issues.append(f"{col}: {conversion_rate:.1f}% giá trị là số (lưu dạng text)")
                print(f"x {col:40s}: {conversion_rate:.1f}% giá trị là số (lưu dạng text)")
        except:
            pass

# 4. Cột hằng số
for col in categorical_cols:
    if df[col].nunique() == 1:
        inappropriate_issues.append(f"{col}: Chỉ có 1 giá trị duy nhất (cột hằng số)")
        print(f"!  {col:40s}: Chỉ có 1 giá trị duy nhất (cột hằng số)")

if not inappropriate_issues:
    print("\n- Tất cả kiểu dữ liệu đều phù hợp!")
else:
    print(f"\n- Tóm tắt: Tìm thấy {len(inappropriate_issues)} vấn đề về kiểu dữ liệu")

!  is_DF                                   : Nên là int, hiện tại là float64
!  is_MF                                   : Nên là int, hiện tại là float64
!  is_FW                                   : Nên là int, hiện tại là float64

- Tóm tắt: Tìm thấy 3 vấn đề về kiểu dữ liệu


#### **5. Phân bố giá trị**

##### **5.1. Các cột số**

In [16]:
if numeric_cols:
    numeric_stats = df[numeric_cols].describe().T
    numeric_stats['độ_lệch'] = [skew(df[col].dropna()) if df[col].notna().any() else np.nan 
                                  for col in numeric_cols]
    numeric_stats['độ_nhọn'] = [kurtosis(df[col].dropna()) if df[col].notna().any() else np.nan 
                                  for col in numeric_cols]
    numeric_stats['thiếu_%'] = [(df[col].isna().sum() / len(df) * 100) for col in numeric_cols]
    
    print("\n", numeric_stats.round(3).to_string())
    
    # Đặc điểm phân bố
    print("ĐẶC ĐIỂM PHÂN BỐ:")
    
    highly_skewed = []
    moderately_skewed = []
    symmetric = []
    
    for col in numeric_cols:
        sk = skew(df[col].dropna())
        if abs(sk) > 2:
            highly_skewed.append((col, sk))
        elif abs(sk) > 0.5:
            moderately_skewed.append((col, sk))
        else:
            symmetric.append((col, sk))
    
    if highly_skewed:
        print(f"\n🔴 Lệch cao (|độ lệch| > 2): {len(highly_skewed)} cột")
        for col, sk in sorted(highly_skewed, key=lambda x: abs(x[1]), reverse=True)[:10]:
            print(f"   • {col:45s}: {sk:7.3f}")
    
    if moderately_skewed:
        print(f"\n🟡 Lệch vừa phải (0.5 < |độ lệch| < 2): {len(moderately_skewed)} cột")
        for col, sk in sorted(moderately_skewed, key=lambda x: abs(x[1]), reverse=True)[:10]:
            print(f"   • {col:45s}: {sk:7.3f}")
    
    if symmetric:
        print(f"\n🟢 Gần đối xứng (|độ lệch| < 0.5): {len(symmetric)} cột")
        for col, sk in symmetric[:5]:
            print(f"   • {col:45s}: {sk:7.3f}")



                                   count      mean      std    min     25%     50%      75%      max  độ_lệch  độ_nhọn  thiếu_%
age                             14977.0    26.505    4.591  16.00   23.00   26.00    30.00    44.00    0.336   -0.379      0.0
appearances                     14977.0    17.682   10.869   0.00    8.00   18.00    27.00    49.00   -0.025   -1.116      0.0
minutes_played                  14977.0  1128.117  889.731   0.00  318.00  994.00  1823.00  4223.00    0.475   -0.827      0.0
is_DF                           14977.0     0.394    0.489   0.00    0.00    0.00     1.00     1.00    0.435   -1.811      0.0
is_MF                           14977.0     0.448    0.497   0.00    0.00    0.00     1.00     1.00    0.210   -1.956      0.0
is_FW                           14977.0     0.158    0.365   0.00    0.00    0.00     0.00     1.00    1.872    1.505      0.0
goals                           14977.0     1.689    2.936   0.00    0.00    1.00     2.00    39.00    3.283 

##### **5.2. Các cột phân loại**

In [17]:
if categorical_cols:
    for col in categorical_cols:
        value_counts = df[col].value_counts()
        n_unique = df[col].nunique()
        
        print(f"\n{col.upper()}")
        print(f"   Số giá trị duy nhất: {n_unique}")
        print(f"   Thiếu: {df[col].isna().sum()} ({df[col].isna().sum()/len(df)*100:.1f}%)")
        print(f"   Top 5 giá trị:")
        for val, count in value_counts.head(5).items():
            pct = count / len(df) * 100
            print(f"      • {str(val):35s}: {count:5d} ({pct:5.1f}%)")


NATIONALITY
   Số giá trị duy nhất: 162
   Thiếu: 0 (0.0%)
   Top 5 giá trị:
      • Argentina                          :  1340 (  8.9%)
      • Brazil                             :  1009 (  6.7%)
      • Japan                              :   860 (  5.7%)
      • Spain                              :   644 (  4.3%)
      • France                             :   523 (  3.5%)

CURRENT_CLUB
   Số giá trị duy nhất: 811
   Thiếu: 0 (0.0%)
   Top 5 giá trị:
      • Independiente                      :    73 (  0.5%)
      • Vitória                            :    52 (  0.3%)
      • Barcelona                          :    50 (  0.3%)
      • Santos                             :    49 (  0.3%)
      • Nacional                           :    41 (  0.3%)

LEAGUE
   Số giá trị duy nhất: 57
   Thiếu: 0 (0.0%)
   Top 5 giá trị:
      • Argentine Liga                     :   945 (  6.3%)
      • La Liga                            :   582 (  3.9%)
      • J1 League                          :   549 

#### **6. Giá trị thiếu**

In [18]:
missing_summary = pd.DataFrame({
    'Cột': df.columns,
    'Thiếu': df.isna().sum().values,
    'Thiếu_%': (df.isna().sum().values / len(df) * 100).round(2),
    'Không_thiếu': df.notna().sum().values
})

missing_summary = missing_summary.sort_values('Thiếu_%', ascending=False)

print(f"\nTóm tắt giá trị thiếu:")
print(f"   - Tổng số ô: {df.shape[0] * df.shape[1]:,}")
print(f"   - Số ô thiếu: {df.isna().sum().sum():,}")
print(f"   - Tỷ lệ thiếu: {(df.isna().sum().sum() / (df.shape[0] * df.shape[1]) * 100):.2f}%")

missing_cols = missing_summary[missing_summary['Thiếu'] > 0]

if len(missing_cols) > 0:
    print(f"\n-  {len(missing_cols)} cột có giá trị thiếu:")
    print("-" * 90)
    print(missing_cols.to_string(index=False))
else:
    print(f"\n- Không có giá trị thiếu trong dataset!")


Tóm tắt giá trị thiếu:
   - Tổng số ô: 629,034
   - Số ô thiếu: 0
   - Tỷ lệ thiếu: 0.00%

- Không có giá trị thiếu trong dataset!


### **III. Kết luận**

### **1. Tổng quan dữ liệu**

**Điểm mạnh:**

- Kích thước phù hợp: Dataset với nhiều bản ghi và đầy đủ thuộc tính
- Chất lượng cao: Tỷ lệ dữ liệu thiếu thấp, cho thấy tiền xử lý đã được thực hiện tốt
- Đa dạng thông tin: Bao gồm đầy đủ các nhóm chỉ số: thông tin cơ bản, tấn công, phòng thủ, chuyền bóng, rê dắt
- Tập trung vào cầu thủ ngoại tuyến: Dataset đã loại bỏ thủ môn để tập trung vào các vị trí ngoài (hậu vệ, tiền vệ, tiền đạo) - những vị trí có chỉ số thống kê phong phú hơn

**Cần xem xét:**

- Phân bố không chuẩn: Biến mục tiêu (market_value) có độ lệch cao
- Nhiều features lệch: Một số cột có độ lệch cao hoặc vừa phải
- Outliers: Một số cầu thủ có giá trị thị trường rất cao (siêu sao) tạo nên outliers

### **2. Kiểu dữ liệu và giá trị**

- Tất cả cột có kiểu dữ liệu phù hợp.

- Các chỉ số nhị phân (is_DF, is_MF, is_FW) đúng format 0/1.

- Giá trị phần trăm nằm trong 0-100%; không có giá trị bất thường.

### **3. Phân bố dữ liệu và outliers**

- Biến mục tiêu market_value lệch phải mạnh (skew = 7.369) do một số siêu sao.

- Một số features khác cũng lệch, cần xử lý trong modeling nếu dùng NN.

- Tree-based models xử lý tốt dữ liệu lệch và outliers.

#### **4. Gợi ý**

- Log transform target để giảm skewness.

- Feature engineering nâng cao (interaction, polynomial, threshold-based) thực hiện trong modeling.

- Lựa chọn thuật toán: Gradient Boosting / Random Forest ưu tiên; Neural Network cần preprocessing kỹ.

- Sử dụng K-Fold Cross-Validation để đảm bảo phân bố đều cho train/test.

#### **5. Kết luận tổng thể**

- Dataset sẵn sàng cho modeling: chất lượng dữ liệu cao, kiểu dữ liệu chính xác, đủ features và samples.

- Cần xử lý target biến lệch và lựa chọn thuật toán phù hợp.

- Insights từ dataset phản ánh thực tế bóng đá: đa số cầu thủ trung bình, một số ít siêu sao tạo outliers.